## Kaggle Challenge - Twitter Sentiment Analysis - Word2Vec Model

- using distributed word vectors created by the Word2Vec algorithm
- Word2vec, published by Google in 2013, is a neural network implementation that learns distributed representations for words. 
- Other deep or recurrent neural network architectures had been proposed for learning word representations prior to this, but the major problem with these was the long time required to train the models. Word2vec learns quickly relative to other models.
- Word2Vec does not need labels in order to create meaningful representations.

### Read the data

In [1]:
import pandas as pd

data = pd.read_csv("../dataset/test.csv", header=0, delimiter=",", encoding='latin-1')
# Word2Vec learns from unlabeled data
# test.csv has far more records (including unlabeled records from train.csv)

In [2]:
data.head()

,ItemID,SentimentText
0,1,is so sad for my APL frie...
1,2,I missed the New Moon trail...
2,3,omg its already 7:30 :O
3,4,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,i think mi bf is cheating on me!!! ...


In [3]:
data.shape

(299989, 2)

In [4]:
data.SentimentText[0]

'                     is so sad for my APL friend.............'

### Clean data
- to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors --> make stop word removal optional in the function

#### Make a specific input format  //// Prepare our data for input to Word2Vec
** Word2Vec - input format is a list of lists **

To make sentences from tweets -> use NLTK's punkt tokenizer for sentence splitting.

In [6]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [8]:
from sentence_splitting import tweet_to_sentences

In [16]:
# Split the training set into clean sentences

sentences = []  # Initialize an empty list of sentences
print("Parsing sentences from training set")
for tweet in data["SentimentText"]:
    sentences += tweet_to_sentences(tweet, tokenizer)

Parsing sentences from training set


/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://myloc.me/27HX" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://bit.ly/azFwv" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://tumblr.com/xkh1z19us" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  

/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://lookbook.nu/look/160916" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://blip.fm/~7df6m" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://blip.fm/~7d2a5" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful So

/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://twitpic.com/6e4a7" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://tinyurl.com/mm9wod" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://tinyurl.com/mlh4f5" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful 

/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://blip.fm/~5il06" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://blip.fm/~7d9p7" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://lookbook.nu/look/154014" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful So

...

/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://myloc.me/4Cal" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://sml.vg/805cbE" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://twitpic.com/7gmr5" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' 

/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://twitpic.com/6vs3b" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://tinyurl.com/murj3f" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://beta.twiddeo.com/8a3!" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautif

/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://tinyurl.com/c3podx" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://blip.fm/~5j85d" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://ohnostop.org?" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  

/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://lookbook.nu/look/143661" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://bit.ly/aRjgA" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://twitpic.com/39nch" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful S

/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://myloc.me/22Ov" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://bit.ly/H9zm6" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/pilot/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://bit.ly/uuGaQ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that d

In [17]:
print(len(sentences))

526997


In [18]:
print(sentences[0])

['be', 'so', 'sad', 'for', 'my', 'apl', 'friend']


### Training and Saving The Model
- With the list of nicely parsed sentences, we're ready to train the model.
- There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation.

In [19]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [20]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size     
downsampling = 1e-3   # Downsample setting for frequent words

In [21]:
# Initialize and train the model
from gensim.models import word2vec
print("Model is training and money is rolling af")
model = word2vec.Word2Vec(sentences, workers=num_workers,size=num_features, min_count=min_word_count,window=context, sample=downsampling)
print("Model is trained...Awesome")

2019-05-18 11:57:03,310 : INFO : 'pattern' package not found; tag filters are not available for English
2019-05-18 11:57:03,380 : INFO : collecting all words and their counts
2019-05-18 11:57:03,381 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-18 11:57:03,416 : INFO : PROGRESS: at sentence #10000, processed 70349 words, keeping 8411 word types
2019-05-18 11:57:03,447 : INFO : PROGRESS: at sentence #20000, processed 154789 words, keeping 15006 word types
2019-05-18 11:57:03,477 : INFO : PROGRESS: at sentence #30000, processed 225062 words, keeping 18680 word types
2019-05-18 11:57:03,506 : INFO : PROGRESS: at sentence #40000, processed 294208 words, keeping 21624 word types
2019-05-18 11:57:03,540 : INFO : PROGRESS: at sentence #50000, processed 365527 words, keeping 23968 word types


Model is training and money is rolling af


2019-05-18 11:57:03,573 : INFO : PROGRESS: at sentence #60000, processed 438311 words, keeping 26289 word types
2019-05-18 11:57:03,608 : INFO : PROGRESS: at sentence #70000, processed 511121 words, keeping 28304 word types
2019-05-18 11:57:03,635 : INFO : PROGRESS: at sentence #80000, processed 583798 words, keeping 30288 word types
2019-05-18 11:57:03,663 : INFO : PROGRESS: at sentence #90000, processed 654690 words, keeping 32096 word types
2019-05-18 11:57:03,695 : INFO : PROGRESS: at sentence #100000, processed 727265 words, keeping 33895 word types
2019-05-18 11:57:03,728 : INFO : PROGRESS: at sentence #110000, processed 801045 words, keeping 35644 word types
2019-05-18 11:57:03,760 : INFO : PROGRESS: at sentence #120000, processed 873847 words, keeping 37244 word types
2019-05-18 11:57:03,805 : INFO : PROGRESS: at sentence #130000, processed 946537 words, keeping 38786 word types
2019-05-18 11:57:03,829 : INFO : PROGRESS: at sentence #140000, processed 1017868 words, keeping 403

2019-05-18 11:57:14,766 : INFO : EPOCH 3 - PROGRESS: at 23.44% examples, 583462 words/s, in_qsize 6, out_qsize 1
2019-05-18 11:57:15,777 : INFO : EPOCH 3 - PROGRESS: at 48.10% examples, 596355 words/s, in_qsize 7, out_qsize 0
2019-05-18 11:57:16,784 : INFO : EPOCH 3 - PROGRESS: at 71.84% examples, 594888 words/s, in_qsize 7, out_qsize 0
2019-05-18 11:57:17,802 : INFO : EPOCH 3 - PROGRESS: at 96.57% examples, 599254 words/s, in_qsize 7, out_qsize 0
2019-05-18 11:57:17,900 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-05-18 11:57:17,913 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-18 11:57:17,920 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-18 11:57:17,921 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-18 11:57:17,922 : INFO : EPOCH - 3 : training on 3827536 raw words (2511303 effective words) took 4.2s, 602435 effective words/s
2019-05-18 11:57:18,953 : INFO : EPOCH 4 - PROG

Model is trained...Awesome


In [22]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

2019-05-18 11:58:28,574 : INFO : precomputing L2-norms of word weight vectors


In [23]:
# сreate a model name and 
# save the model for later use. to load it later use Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-05-18 11:58:50,361 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2019-05-18 11:58:50,363 : INFO : not storing attribute vectors_norm
2019-05-18 11:58:50,366 : INFO : not storing attribute cum_table
2019-05-18 11:58:50,367 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-18 11:58:50,513 : INFO : saved 300features_40minwords_10context


### Check out the created model /// Exploring the Model Results

In [24]:
model.doesnt_match("man woman child kitchen".split())

/home/pilot/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/home/pilot/anaconda3/lib/python3.6/site-packages/gensim/models/keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [27]:
# well, dataset is too small, so ...
model.doesnt_match("france england germany berlin".split())

/home/pilot/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  
/home/pilot/anaconda3/lib/python3.6/site-packages/gensim/models/keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'

In [29]:
model.wv.most_similar("man")

[('gosh', 0.5753857493400574),
 ('boy', 0.5555647611618042),
 ('dude', 0.5509142875671387),
 ('bugger', 0.4855022430419922),
 ('noes', 0.4791525900363922),
 ('lord', 0.47809916734695435),
 ('pup', 0.4734112620353699),
 ('girl', 0.4489648938179016),
 ('bitch', 0.4437384009361267),
 ('baby', 0.4411129355430603)]

In [31]:
model.wv.most_similar("queen")

[('king', 0.716083824634552),
 ('kings', 0.6854082345962524),
 ('champion', 0.6826828718185425),
 ('beauty', 0.635041356086731),
 ('ultimate', 0.6294366121292114),
 ('youth', 0.6240695714950562),
 ('valley', 0.6143238544464111),
 ('princess', 0.6062234044075012),
 ('celebration', 0.6030943989753723),
 ('castle', 0.599313497543335)]

In [33]:
model.wv.most_similar("awful")

[('terrible', 0.7777630090713501),
 ('horrible', 0.7680115103721619),
 ('harsh', 0.6576536893844604),
 ('disgust', 0.6514105796813965),
 ('awkward', 0.6496937274932861),
 ('intense', 0.6449944972991943),
 ('painful', 0.6346228122711182),
 ('scary', 0.619954526424408),
 ('weird', 0.6164208650588989),
 ('gross', 0.612415075302124)]

Our model is capable of distinguishing differences in meaning! **The model is trained to understand semantic understanding of words**

#### How can we use these fancy distributed word vectors for supervised learning?

### Numeric Representations of Words

Docs - Word2Vec model has a feature vector for each word in the vocabulary, stored in a numpy array called "syn0" - updated - "model.wv.vectors"

In [34]:
type(model.wv.vectors)

numpy.ndarray

In [38]:
model.wv.vectors.shape
# The number of rows in syn0 is the number of words in the model's vocabulary, 
# and the number of columns corresponds to the size of the feature vector, which was set while training model

(4101, 300)

Setting the minimum word count to 40 gave us a total vocabulary of 4101 words with 300 features apiece.

In [46]:
# Individual word vectors can be accessed
model.wv.__getitem__("flower") # model["flower"]

array([ 7.53886700e-02, -1.03168143e-02, -2.17584675e-04, -1.27709229e-02,
       -9.08083841e-02,  2.33558044e-02, -9.92260277e-02,  1.77289657e-02,
       -4.02723290e-02,  1.68949645e-02,  1.24585122e-01,  7.86036029e-02,
        3.85479294e-02, -1.70123298e-02, -3.38637363e-03,  1.06991969e-01,
        6.71893284e-02, -9.18496624e-02, -4.46928479e-03,  1.81568582e-02,
       -1.50800152e-02,  6.21909462e-02, -8.60433578e-02, -1.13667481e-01,
        6.27540238e-03, -3.87985073e-02,  2.28160340e-02,  7.26828724e-03,
        1.22808190e-02, -2.31342837e-02,  4.21061908e-04,  8.16268027e-02,
        1.87841791e-03, -1.36375770e-01,  6.68246076e-02,  8.75561237e-02,
        4.78768907e-02,  5.84915932e-03, -4.84957621e-02, -5.21859862e-02,
        9.03712958e-03, -3.58745456e-02, -6.72433376e-02, -1.07319601e-01,
       -3.86736244e-02,  5.12074605e-02, -4.54497291e-03,  2.03655288e-02,
        6.20704815e-02,  4.40211035e-03, -4.54627350e-02,  1.55422352e-02,
       -7.01879933e-02,  

In [49]:
len(model.wv.__getitem__("flower"))

300

In [52]:
words = model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size",vocab_size)

Vocab size 4101


### From Words To Vectors, Attempt 1: Vector Averaging

- Calculate average feature vectors for training and testing sets

One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

**Create average vectors for each paragraph**

In [55]:
from cleanData_word2vec import tweet_to_wordlist

clean_train_tweets = []
for tweet in data["SentimentText"]:
    clean_train_tweets.append(tweet_to_wordlist(tweet, remove_stopwords=True))

In [59]:
from AverageVectors import getAvgFeatureVecs

In [60]:
trainDataVecs = getAvgFeatureVecs(clean_train_tweets, model, num_features)

Review 0.0 of 299989


/home/pilot/bachelorDeath/Sentiment-Analysis/word2vec_model/AverageVectors.py:26: RuntimeWarning: invalid value encountered in true_divide
  featureVec = np.divide(featureVec,nwords)


Review 1000.0 of 299989
Review 2000.0 of 299989
Review 3000.0 of 299989
Review 4000.0 of 299989
Review 5000.0 of 299989
Review 6000.0 of 299989
Review 7000.0 of 299989
Review 8000.0 of 299989
Review 9000.0 of 299989
Review 10000.0 of 299989
Review 11000.0 of 299989
Review 12000.0 of 299989
Review 13000.0 of 299989
Review 14000.0 of 299989
Review 15000.0 of 299989
Review 16000.0 of 299989
Review 17000.0 of 299989
Review 18000.0 of 299989
Review 19000.0 of 299989
Review 20000.0 of 299989
Review 21000.0 of 299989
Review 22000.0 of 299989
Review 23000.0 of 299989
Review 24000.0 of 299989
Review 25000.0 of 299989
Review 26000.0 of 299989
Review 27000.0 of 299989
Review 28000.0 of 299989
Review 29000.0 of 299989
Review 30000.0 of 299989
Review 31000.0 of 299989
Review 32000.0 of 299989
Review 33000.0 of 299989
Review 34000.0 of 299989
Review 35000.0 of 299989
Review 36000.0 of 299989
Review 37000.0 of 299989
Review 38000.0 of 299989
Review 39000.0 of 299989
Review 40000.0 of 299989
Review 41

In [62]:
len(trainDataVecs[0])

300

**Use the average paragraph vectors to train a random forest.**

In [63]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

/home/pilot/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [64]:
#print("Fitting a random forest to labeled training data...")
#forest = forest.fit( trainDataVecs, train["sentiment"] )

In [65]:
# Test & extract results 
#result = forest.predict(testDataVecs)

In [66]:
# Write the test results 
#output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
#output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting=3)

This produced results underperformed Bag of Words by a few percentage points.

A standard way of weighting word vectors is to apply "tf-idf" weights, which measure how important a given word is within a given set of documents. However, when we tried weighting our word vectors in this way, we found no substantial improvement in performance.

### From Words to Vectors, Attempt 2: Clustering

Word2Vec creates clusters of semantically related words, so another possible approach is to exploit the similarity of words within a cluster.

Grouping vectors in this way is known as "vector quantization." To accomplish this, we first need to find the centers of the word clusters, which we can do by using a clustering algorithm such as K-Means.

From Tutorials - In K-Means, the one parameter we need to set is "K," or the number of clusters. How should we decide how many clusters to create? Trial and error suggested that small clusters, with an average of only 5 words or so per cluster, gave better results than large clusters with many words. Clustering code is given below.

- K-Means clustering with large K can be very slow;
- set a timer around the K-Means function to see how long it takes.

In [68]:
from sklearn.cluster import KMeans

In [70]:
import time
start = time.time() # Start time

In [81]:
# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.vectors            # a feature vector for each word in the vocabulary
num_clusters = int(word_vectors.shape[0] / 5)

In [83]:
# Initalize a k-means object and use it to extract centroids
print("Running K means")
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, " seconds.")

Running K means
Time taken for K Means clustering:  670.0179200172424  seconds.


The cluster assignment for each word is now stored in idx, and the vocabulary from original Word2Vec model is still stored in model.index2word

In [85]:
# Create a Word / Index dictionary, mapping each vocabulary word to a cluster number                                                                                            
word_centroid_map = dict(zip(model.wv.index2word, idx))

In [100]:
# what clusters contain
# For the first 10 clusters
clustersNum = list(word_centroid_map.values())
clustersWord = list(word_centroid_map.keys())

for cluster in range(0,10): 
    print("\nCluster", cluster)

    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if(clustersNum[i] == cluster ):
            words.append(clustersWord[i])
    print(words)
# well, clusters are kind of fuzzy


Cluster 0
['ish']

Cluster 1
['ap']

Cluster 2
['youu', 'youuu', 'youuuu', 'terribly', 'awwwwww', 'yaa', 'yous']

Cluster 3
['paint', 'punch', 'flip', 'smash', 'lift', 'pin', 'spray']

Cluster 4
['blonde']

Cluster 5
['cafe', 'museum', 'lounge']

Cluster 6
['vs', 'comic', 'comedy', 'cartoon', 'theater', 'hp', 'horror', 'eurovision', 'csi', 'urban', 'eminem', 'anime', 'sytycd']

Cluster 7
['kay', 'dw']

Cluster 8
['ouch', 'poo', 'yuck']

Cluster 9
['update', 'iphone', 'account', 'app', 'stream', 'mobile', 'connection', 'error', 'client', 'browser']


### Create bags of centroids

In [103]:
import numpy as np

At any rate, now there is a cluster (or "centroid") assignment for each word, and we can define a function to convert tweets into bags-of-centroids.

In [104]:
def create_bag_of_centroids(wordlist, word_centroid_map):

    # The number of clusters is equal to the highest cluster index in the word / centroid map
    num_centroids = max(word_centroid_map.values()) + 1
    
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    
    # Loop over the words in the tweet. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    # return numpy array for each tweet
    return bag_of_centroids

Create bags of centroids for training set, then train a random forest and extract results

In [105]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros((data["SentimentText"].size, num_clusters), dtype="float32")

In [106]:
# Transform the training set reviews into bags of centroids
counter = 0
for twitt in clean_train_tweets:
    train_centroids[counter] = create_bag_of_centroids(twitt, word_centroid_map)
    counter += 1

In [107]:
# Fit a random forest and extract predictions 
# forest = RandomForestClassifier(n_estimators = 100)

In [108]:
# forest = forest.fit(train_centroids,train["sentiment"])
# result = forest.predict(test_centroids)

In [ ]:
# Write the test results 
# output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
# output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )